<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/tutorials/3-Practical/FinRL_PaperTrading_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Disclaimer: Nothing herein is financial advice, and NOT a recommendation to trade real money. Many platforms exist for simulated trading (paper trading) which can be used for building and developing the methods discussed. Please use common sense and always first consult a professional before trading or investing.

# Part 1: Install FinRL

In [ ]:
## install finrl library
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-od0ekoy6
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-od0ekoy6
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-05_agdff/pyfolio_d9554690934d499785fcab16dd5cac5c
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-05_agdff/pyfolio_d9554690934d499785fcab16dd5cac5c
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-05_agdff/elegantrl_7154d79555e74afb85c897a1b9679942
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-05_agdff/elegantrl_7154d79555e74afb85c897a1b9679942
     |████████████████████████████████| 73 kB 1.7 MB/s 
     |████████████████████████████████| 174 kB 11.4 MB/s 
     |████████████████████████████████| 54.7 MB 1.4 MB/s 
     |████████████████████████████████| 1.8 MB 37.8 MB/s 
     |██████

## Import related modules

In [1]:
from finrl.train import train
from finrl.test import test
from finrl.config_tickers import DOW_30_TICKER
from finrl.config import INDICATORS
from finrl.finrl_meta.env_stock_trading.env_stocktrading_np import StockTradingEnv
from finrl.finrl_meta.env_stock_trading.env_stock_papertrading import AlpacaPaperTrading
from finrl.finrl_meta.data_processor import DataProcessor
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

import numpy as np
import pandas as pd

/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


## Import Dow Jones 30 Symbols

In [2]:
ticker_list = DOW_30_TICKER
action_dim = len(DOW_30_TICKER)
candle_time_interval = '15Min'  # '1Min'

In [3]:
print(ticker_list)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [4]:
print(INDICATORS)

['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']


## Calculate the DRL state dimension manually for paper trading

In [5]:
# amount + (turbulence, turbulence_bool) + (price, shares, cd (holding time)) * stock_dim + tech_dim
state_dim = 1 + 2 + 3 * action_dim + len(INDICATORS) * action_dim

In [6]:
state_dim

333

## Get the API Keys Ready

In [7]:
API_KEY = "PKC45PE857XWAGUVR7WK"
API_SECRET = "e8v8RrHpZK4ZHhyyh9OfeNygu9FjAizJRVai93Oh"
API_BASE_URL = 'https://paper-api.alpaca.markets'
data_url = 'wss://data.alpaca.markets'
env = StockTradingEnv

## Show the data

### Step 1. Pick a data source

In [8]:
DP = DataProcessor(data_source = 'alpaca',
                  API_KEY = API_KEY, 
                  API_SECRET = API_SECRET, 
                  API_BASE_URL = API_BASE_URL
                  )

Alpaca successfully connected


### Step 2. Get ticker list, Set start date and end date, specify the data frequency

In [31]:
data = DP.download_data(start_date = '2021-10-01', 
                        end_date = '2021-10-05',
                        ticker_list = ticker_list, 
                        time_interval= candle_time_interval)

Data before 2021-10-01T15:59:00-04:00 is successfully fetched
Data before 2021-10-02T15:59:00-04:00 is successfully fetched
Data before 2021-10-03T15:59:00-04:00 is successfully fetched
Data before 2021-10-04T15:59:00-04:00 is successfully fetched
Data before 2021-10-05T15:59:00-04:00 is successfully fetched


### Step 3. Data Cleaning & Feature Engineering

In [32]:
data = DP.clean_data(data)
data = DP.add_technical_indicator(data, INDICATORS)
data = DP.add_vix(data)

The price of the first row for ticker  AAPL  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  AMGN  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  AXP  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  BA  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CAT  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CRM  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CSCO  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CVX  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  DIS  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  DOW  is NaN.  It will filled with the first valid price.
The price of the first row for ticker 

In [25]:
data.head(20)

,time,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,VIXY
0,2021-10-01 09:30:00-04:00,141.8900,141.8900,141.8900,141.8900,0.0,AAPL,0.0,NaN,NaN,NaN,NaN,NaN,141.8900,141.8900,22.17
1,2021-10-01 09:30:00-04:00,212.7100,212.7100,212.7100,212.7100,0.0,AMGN,0.0,NaN,NaN,NaN,NaN,NaN,212.7100,212.7100,22.17
2,2021-10-01 09:30:00-04:00,168.8600,168.8600,168.8600,168.8600,0.0,AXP,0.0,NaN,NaN,NaN,NaN,NaN,168.8600,168.8600,22.17
3,2021-10-01 09:30:00-04:00,223.7700,223.7700,223.7700,223.7700,0.0,BA,0.0,NaN,NaN,NaN,NaN,NaN,223.7700,223.7700,22.17
4,2021-10-01 09:30:00-04:00,193.7300,193.7300,193.7300,193.7300,0.0,CAT,0.0,NaN,NaN,NaN,NaN,NaN,193.7300,193.7300,22.17
5,2021-10-01 09:30:00-04:00,272.9750,272.9750,272.9750,272.9750,0.0,CRM,0.0,NaN,NaN,NaN,NaN,NaN,272.9750,272.9750,22.17
6,2021-10-01 09:30:00-04:00,54.5400,54.5400,54.5400,54.5400,0.0,CSCO,0.0,NaN,NaN,NaN,NaN,NaN,54.5400,54.5400,22.17
7,2021-10-01 09:30:00-04:00,102.4500,102.4500,102.4500,102.4500,0.0,CVX,0.0,NaN,NaN,NaN,NaN,NaN,102.4500,102.4500,22.17
8,2021-10-01 09:30:00-04:00,172.9500,172.9500,172.9500,172.9500,0.0,DIS,0.0,NaN,NaN,NaN,NaN,NaN,172.9500,172.9500,22.17
9,2021-10-01 09:30:00-04:00,57.9600,57.9600,57.9600,57.9600,0.0,DOW,0.0,NaN,NaN,NaN,NaN,NaN,57.9600,57.9600,22.17


In [26]:
data.tail(20)

,time,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,VIXY
65572,2021-10-05 19:59:00,386.740,386.79,385.7900,386.02,83446.0,GS,-0.193803,387.627940,386.404800,37.677047,-355.200649,45.061599,387.070840,387.360087,21.79
65573,2021-10-05 19:59:00,329.680,329.74,329.3000,329.58,48366.0,HD,-0.422900,331.596256,329.381484,29.668209,-159.256745,75.111333,330.791143,331.197633,21.79
65574,2021-10-05 19:59:00,215.740,215.74,215.5900,215.72,68912.0,HON,-0.224308,217.025524,215.602286,28.510098,-181.500168,70.107704,216.421770,216.523110,21.79
65575,2021-10-05 19:59:00,143.140,143.22,143.0600,143.16,161386.0,IBM,-0.144848,143.861854,143.058786,32.811758,-165.677474,65.938499,143.531590,143.718792,21.79
65576,2021-10-05 19:59:00,53.955,53.98,53.9200,53.95,402637.0,INTC,-0.038752,54.169893,53.923087,33.417829,-171.069248,53.406043,54.059623,54.095910,21.79
65577,2021-10-05 19:59:00,159.740,159.74,159.5200,159.57,130752.0,JNJ,-0.072861,160.079432,159.650368,35.875310,-314.385924,34.523986,159.886600,160.017605,21.79
65578,2021-10-05 19:59:00,168.880,168.90,168.6200,168.65,188762.0,JPM,-0.137473,169.502615,168.673755,33.961213,-172.093998,57.964591,169.178113,169.314243,21.79
65579,2021-10-05 19:59:00,53.085,53.09,53.0500,53.09,373417.0,KO,-0.011948,53.195412,53.073918,43.524905,-190.205011,31.234146,53.139610,53.139403,21.79
65580,2021-10-05 19:59:00,245.290,245.41,245.1500,245.17,63342.0,MCD,-0.268791,246.654587,245.079583,25.269499,-160.909519,78.409744,246.034130,246.176732,21.79
65581,2021-10-05 19:59:00,177.960,177.96,177.7800,177.88,52055.0,MMM,-0.082925,178.480310,177.875190,39.572766,-202.728603,56.466497,178.229567,178.194567,21.79


In [33]:
for tic in data.tic.unique():
    length = data[data.tic == tic][["close"]].values.shape
    print(tic, length)

AAPL (1238, 1)
AMGN (1238, 1)
AXP (1238, 1)
BA (1238, 1)
CAT (1238, 1)
CRM (1238, 1)
CSCO (1238, 1)
CVX (1238, 1)
DIS (1238, 1)
DOW (1238, 1)
GS (1238, 1)
HD (1238, 1)
HON (1238, 1)
IBM (1238, 1)
INTC (1238, 1)
JNJ (1238, 1)
JPM (1238, 1)
KO (1238, 1)
MCD (1238, 1)
MMM (1238, 1)
MRK (1238, 1)
MSFT (1238, 1)
NKE (1238, 1)
PG (1238, 1)
TRV (1238, 1)
UNH (1238, 1)
V (1238, 1)
VZ (1238, 1)
WBA (1238, 1)
WMT (1238, 1)


### Step 4. Transform to numpy array

In [34]:
price_array, tech_array, turbulence_array = DP.df_to_array(data, if_vix='True')

Successfully transformed into array


In [36]:
print(price_array.shape, tech_array.shape, turbulence_array.shape)

(1238, 30) (1238, 240) (1238,)


# Part 2: Train the agent

## Train

In [80]:
ERL_PARAMS = {"learning_rate": 3e-6,"batch_size": 2048,"gamma":  0.985,
        "seed":312,"net_dimension":512, "target_step":5000, "eval_gap":30,
        "eval_times":1} 
#if you want to use larger datasets (change to longer period), and it raises error, 
#please try to increase "target_step". It should be larger than the episode steps. 

In [81]:
train(start_date = '2021-10-11', 
      end_date = '2021-10-15',
      ticker_list = ticker_list, 
      data_source = 'alpaca',
      time_interval= candle_time_interval, 
      technical_indicator_list= INDICATORS,
      drl_lib='elegantrl', 
      env=env,
      model_name='ppo', 
      API_KEY = API_KEY, 
      API_SECRET = API_SECRET, 
      API_BASE_URL = API_BASE_URL,
      erl_params=ERL_PARAMS,
      cwd='./papertrading_erl', #current_working_dir
      break_step=1e5)



Alpaca successfully connected
Data before 2021-10-11T15:59:00-04:00 is successfully fetched
Data before 2021-10-12T15:59:00-04:00 is successfully fetched
Data before 2021-10-13T15:59:00-04:00 is successfully fetched
Data before 2021-10-14T15:59:00-04:00 is successfully fetched
Data before 2021-10-15T15:59:00-04:00 is successfully fetched
The price of the first row for ticker  AAPL  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  AMGN  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  AXP  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  BA  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CAT  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CRM  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CSCO  is NaN.  It will filled with the first

ModuleNotFoundError: No module named 'finrl.agents'

## Test

In [82]:
account_value_erl=test(start_date = '2021-10-18', 
                      end_date = '2021-10-19',
                      ticker_list = ticker_list, 
                      data_source = 'alpaca',
                      time_interval= candle_time_interval, 
                      technical_indicator_list= INDICATORS,
                      drl_lib='elegantrl', 
                      env=env, 
                      model_name='ppo', 
                      API_KEY = API_KEY, 
                      API_SECRET = API_SECRET, 
                      API_BASE_URL = API_BASE_URL,
                      cwd='./papertrading_erl',
                      net_dimension = 512)

Alpaca successfully connected
Data before 2021-10-18T15:59:00-04:00 is successfully fetched
Data before 2021-10-19T15:59:00-04:00 is successfully fetched
The price of the first row for ticker  AAPL  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  AMGN  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  AXP  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  BA  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CAT  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CRM  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CSCO  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  CVX  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  DIS  is NaN.  It will filled with the

ModuleNotFoundError: No module named 'finrl.agents'

## Use full data to train 

After tuning well, retrain on the training and testing sets

In [ ]:
train(start_date = '2021-10-11', 
      end_date = '2021-10-19',
      ticker_list = ticker_list, 
      data_source = 'alpaca',
      time_interval= candle_time_interval, 
      technical_indicator_list= INDICATORS,
      drl_lib='elegantrl', 
      env=env, 
      model_name='ppo', 
      API_KEY = API_KEY, 
      API_SECRET = API_SECRET, 
      API_BASE_URL = API_BASE_URL,
      erl_params=ERL_PARAMS,
      cwd='./papertrading_erl_retrain',
      break_step=5e4)

Alpaca successfully connected
Data before 2021-10-11T15:59:00-04:00 is successfully fetched
Data before 2021-10-12T15:59:00-04:00 is successfully fetched
Data before 2021-10-13T15:59:00-04:00 is successfully fetched
Data before 2021-10-14T15:59:00-04:00 is successfully fetched
Data before 2021-10-15T15:59:00-04:00 is successfully fetched
Data before 2021-10-16T15:59:00-04:00 is successfully fetched
Data before 2021-10-17T15:59:00-04:00 is successfully fetched


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1Min 3 more time(s)...


Data before 2021-10-18T15:59:00-04:00 is successfully fetched


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1Min 3 more time(s)...


Data before 2021-10-19T15:59:00-04:00 is successfully fetched
Data clean finished!
Succesfully add technical indicators
Data before 2021-10-11T15:59:00-04:00 is successfully fetched
Data before 2021-10-12T15:59:00-04:00 is successfully fetched
Data before 2021-10-13T15:59:00-04:00 is successfully fetched
Data before 2021-10-14T15:59:00-04:00 is successfully fetched
Data before 2021-10-15T15:59:00-04:00 is successfully fetched
Data before 2021-10-16T15:59:00-04:00 is successfully fetched
Data before 2021-10-17T15:59:00-04:00 is successfully fetched
Data before 2021-10-18T15:59:00-04:00 is successfully fetched
Data before 2021-10-19T15:59:00-04:00 is successfully fetched
Data clean finished!
Successfully transformed into array
| Remove cwd: ./papertrading_erl_retrain
################################################################################
ID     Step    maxR |    avgR   stdR   avgS  stdS |    expR   objC   etc.
0  2.73e+03    1.01 |
0  2.73e+03    1.01 |    1.01    0.0   2728    

# Part 3: Deploy the agent

## Setup Alpaca Paper trading environment

In [9]:
import datetime
import threading
from finrl.finrl_meta.data_processors.processor_alpaca import AlpacaProcessor
import alpaca_trade_api as tradeapi
import time
import pandas as pd
import numpy as np
import torch
import gym

class AlpacaPaperTrading():

    def __init__(self,ticker_list, time_interval, drl_lib, agent, cwd, net_dim, 
                 state_dim, action_dim, API_KEY, API_SECRET, 
                 API_BASE_URL, tech_indicator_list, turbulence_thresh=30, 
                 max_stock=1e2, latency = None):
        #load agent
        self.drl_lib = drl_lib
        if agent =='ppo':
            if drl_lib == 'elegantrl':              
                from elegantrl.agents import AgentPPO
                from elegantrl.train.run import init_agent
                from elegantrl.train.config import Arguments
                #load agent
                config = {'state_dim':state_dim,
                            'action_dim':action_dim,}
                args = Arguments(agent=AgentPPO, env=StockEnvEmpty(config))
                args.cwd = cwd
                args.net_dim = net_dim
                # load agent
                try:
                    agent = init_agent(args, gpu_id = 0)
                    self.act = agent.act
                    self.device = agent.device
                except BaseException:
                    raise ValueError("Fail to load agent!")
                        
            elif drl_lib == 'rllib':
                from ray.rllib.agents import ppo
                from ray.rllib.agents.ppo.ppo import PPOTrainer
                
                config = ppo.DEFAULT_CONFIG.copy()
                config['env'] = StockEnvEmpty
                config["log_level"] = "WARN"
                config['env_config'] = {'state_dim':state_dim,
                            'action_dim':action_dim,}
                trainer = PPOTrainer(env=StockEnvEmpty, config=config)
                trainer.restore(cwd)
                try:
                    trainer.restore(cwd)
                    self.agent = trainer
                    print("Restoring from checkpoint path", cwd)
                except:
                    raise ValueError('Fail to load agent!')
                    
            elif drl_lib == 'stable_baselines3':
                from stable_baselines3 import PPO
                
                try:
                    #load agent
                    self.model = PPO.load(cwd)
                    print("Successfully load model", cwd)
                except:
                    raise ValueError('Fail to load agent!')
                    
            else:
                raise ValueError('The DRL library input is NOT supported yet. Please check your input.')
               
        else:
            raise ValueError('Agent input is NOT supported yet.')
            
            
            
        #connect to Alpaca trading API
        try:
            self.alpaca = tradeapi.REST(API_KEY,API_SECRET,API_BASE_URL, 'v2')
        except:
            raise ValueError('Fail to connect Alpaca. Please check account info and internet connection.')
        
        #read trading time interval
        if time_interval == '1s':
            self.time_interval = 1
        elif time_interval == '5s':
            self.time_interval = 5
        elif time_interval == '1Min':
            self.time_interval = 60
        elif time_interval == '5Min':
            self.time_interval = 60 * 5
        elif time_interval == '15Min':
            self.time_interval = 60 * 15
        else:
            raise ValueError('Time interval input is NOT supported yet.')
        
        #read trading settings
        self.tech_indicator_list = tech_indicator_list
        self.turbulence_thresh = turbulence_thresh
        self.max_stock = max_stock 
        
        #initialize account
        self.stocks = np.asarray([0] * len(ticker_list)) #stocks holding
        self.stocks_cd = np.zeros_like(self.stocks) 
        self.cash = None #cash record 
        self.stocks_df = pd.DataFrame(self.stocks, columns=['stocks'], index = ticker_list)
        self.asset_list = []
        self.price = np.asarray([0] * len(ticker_list))
        self.stockUniverse = ticker_list
        self.turbulence_bool = 0
        self.equities = []
        
    def test_latency(self, test_times = 10): 
        total_time = 0
        for i in range(0, test_times):
            time0 = time.time()
            self.get_state()
            time1 = time.time()
            temp_time = time1 - time0
            total_time += temp_time
        latency = total_time/test_times
        print('latency for data processing: ', latency)
        return latency
        
    def run(self):
        orders = self.alpaca.list_orders(status="open")
        for order in orders:
          self.alpaca.cancel_order(order.id)
    
        # Wait for market to open.
        print("Waiting for market to open...")
        tAMO = threading.Thread(target=self.awaitMarketOpen)
        tAMO.start()
        tAMO.join()
        print("Market opened.")
        while True:

          # Figure out when the market will close so we can prepare to sell beforehand.
          clock = self.alpaca.get_clock()
          closingTime = clock.next_close.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          self.timeToClose = closingTime - currTime
    
          if(self.timeToClose < (60)):
            # Close all positions when 1 minutes til market close.
            print("Market closing soon. Stop trading.")
            break
            
            '''# Close all positions when 1 minutes til market close.
            print("Market closing soon.  Closing positions.")
    
            positions = self.alpaca.list_positions()
            for position in positions:
              if(position.side == 'long'):
                orderSide = 'sell'
              else:
                orderSide = 'buy'
              qty = abs(int(float(position.qty)))
              respSO = []
              tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
              tSubmitOrder.start()
              tSubmitOrder.join()
    
            # Run script again after market close for next trading day.
            print("Sleeping until market close (15 minutes).")
            time.sleep(60 * 15)'''
            
          else:
            trade = threading.Thread(target=self.trade)
            trade.start()
            trade.join()
            last_equity = float(self.alpaca.get_account().last_equity)
            cur_time = time.time()
            self.equities.append([cur_time,last_equity])
            time.sleep(self.time_interval)
            
    def awaitMarketOpen(self):
        isOpen = self.alpaca.get_clock().is_open
        while(not isOpen):
          clock = self.alpaca.get_clock()
          openingTime = clock.next_open.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          timeToOpen = int((openingTime - currTime) / 60)
          print(str(timeToOpen) + " minutes til market open.")
          time.sleep(60)
          isOpen = self.alpaca.get_clock().is_open
    
    def trade(self):
        state = self.get_state()
        
        if self.drl_lib == 'elegantrl':
            with torch.no_grad():
                s_tensor = torch.as_tensor((state,), device=self.device)
                a_tensor = self.act(s_tensor)  
                action = a_tensor.detach().cpu().numpy()[0]  
                
            action = (action * self.max_stock).astype(int)
            
        elif self.drl_lib == 'rllib':
            action = self.agent.compute_single_action(state)
        
        elif self.drl_lib == 'stable_baselines3':
            action = self.model.predict(state)[0]
            
        else:
            raise ValueError('The DRL library input is NOT supported yet. Please check your input.')
        
        self.stocks_cd += 1
        if self.turbulence_bool == 0:
            min_action = 10  # stock_cd
            for index in np.where(action < -min_action)[0]:  # sell_index:
                sell_num_shares = min(self.stocks[index], -action[index])
                qty =  abs(int(sell_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'sell', respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0

            for index in np.where(action > min_action)[0]:  # buy_index:
                if self.cash < 0:
                    tmp_cash = 0
                else:
                    tmp_cash = self.cash
                buy_num_shares = min(tmp_cash // self.price[index], abs(int(action[index])))
                qty = abs(int(buy_num_shares))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, self.stockUniverse[index], 'buy', respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
                self.cash = float(self.alpaca.get_account().cash)
                self.stocks_cd[index] = 0
                
        else:  # sell all when turbulence
            positions = self.alpaca.list_positions()
            for position in positions:
                if(position.side == 'long'):
                    orderSide = 'sell'
                else:
                    orderSide = 'buy'
                qty = abs(int(float(position.qty)))
                respSO = []
                tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
                tSubmitOrder.start()
                tSubmitOrder.join()
            
            self.stocks_cd[:] = 0
            
    
    def get_state(self):
        alpaca = AlpacaProcessor(api=self.alpaca)
        price, tech, turbulence = alpaca.fetch_latest_data(ticker_list = self.stockUniverse, time_interval='1Min',
                                                     tech_indicator_list=self.tech_indicator_list)
        turbulence_bool = 1 if turbulence >= self.turbulence_thresh else 0
        
        turbulence = (self.sigmoid_sign(turbulence, self.turbulence_thresh) * 2 ** -5).astype(np.float32)
        
        tech = tech * 2 ** -7
        positions = self.alpaca.list_positions()
        stocks = [0] * len(self.stockUniverse)
        for position in positions:
            ind = self.stockUniverse.index(position.symbol)
            stocks[ind] = ( abs(int(float(position.qty))))
        
        stocks = np.asarray(stocks, dtype = float)
        cash = float(self.alpaca.get_account().cash)
        self.cash = cash
        self.stocks = stocks
        self.turbulence_bool = turbulence_bool 
        self.price = price
        
        
        
        amount = np.array(self.cash * (2 ** -12), dtype=np.float32)
        scale = np.array(2 ** -6, dtype=np.float32)
        state = np.hstack((amount,
                    turbulence,
                    self.turbulence_bool,
                    price * scale,
                    self.stocks * scale,
                    self.stocks_cd,
                    tech,
                    )).astype(np.float32)
        print(len(self.stockUniverse))
        return state
        
    def submitOrder(self, qty, stock, side, resp):
        if(qty > 0):
          try:
            self.alpaca.submit_order(stock, qty, side, "market", "day")
            print("Market order of | " + str(qty) + " " + stock + " " + side + " | completed.")
            resp.append(True)
          except:
            print("Order of | " + str(qty) + " " + stock + " " + side + " | did not go through.")
            resp.append(False)
        else:
          print("Quantity is 0, order of | " + str(qty) + " " + stock + " " + side + " | not completed.")
          resp.append(True)

    @staticmethod
    def sigmoid_sign(ary, thresh):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x * np.e)) - 0.5

        return sigmoid(ary / thresh) * thresh
    
class StockEnvEmpty(gym.Env):
    #Empty Env used for loading rllib agent
    def __init__(self,config):
      state_dim = config['state_dim']
      action_dim = config['action_dim']
      self.env_num = 1
      self.max_step = 10000
      self.env_name = 'StockEnvEmpty'
      self.state_dim = state_dim  
      self.action_dim = action_dim
      self.if_discrete = False  
      self.target_return = 9999
      self.observation_space = gym.spaces.Box(low=-3000, high=3000, shape=(state_dim,), dtype=np.float32)
      self.action_space = gym.spaces.Box(low=-1, high=1, shape=(action_dim,), dtype=np.float32)
        
    def reset(self):
        return 

    def step(self, actions):
        return

## Run Paper trading

In [10]:
print(DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [11]:
state_dim

333

In [12]:
action_dim

30

In [13]:
paper_trading_erl = AlpacaPaperTrading(ticker_list = DOW_30_TICKER, 
                                       time_interval = '1Min', 
                                       drl_lib = 'elegantrl', 
                                       agent = 'ppo', 
                                       cwd = './papertrading_erl_retrain', 
                                       net_dim = 512, 
                                       state_dim = state_dim, 
                                       action_dim= action_dim, 
                                       API_KEY = API_KEY, 
                                       API_SECRET = API_SECRET, 
                                       API_BASE_URL = API_BASE_URL, 
                                       tech_indicator_list = INDICATORS, 
                                       turbulence_thresh=30, 
                                       max_stock=1e2)
paper_trading_erl.run()

Waiting for market to open...
Market opened.
Succesfully add technical indicators
Successfully transformed into array
30


/home/quantumiracle/anaconda3/envs/x/lib/python3.7/site-packages/ipykernel_launcher.py:190: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)


Order of | 9223372036854775808 AXP sell | did not go through.
Order of | 9223372036854775808 AMGN sell | did not go through.
Order of | 9223372036854775808 AAPL sell | did not go through.
Order of | 9223372036854775808 BA sell | did not go through.
Order of | 9223372036854775808 CAT sell | did not go through.
Order of | 9223372036854775808 CSCO sell | did not go through.
Order of | 9223372036854775808 CVX sell | did not go through.
Order of | 9223372036854775808 GS sell | did not go through.
Order of | 9223372036854775808 HD sell | did not go through.
Order of | 9223372036854775808 HON sell | did not go through.
Order of | 9223372036854775808 IBM sell | did not go through.
Order of | 9223372036854775808 INTC sell | did not go through.
Order of | 9223372036854775808 JNJ sell | did not go through.
Order of | 9223372036854775808 KO sell | did not go through.
Order of | 9223372036854775808 JPM sell | did not go through.
Order of | 9223372036854775808 MCD sell | did not go through.
Order of

KeyboardInterrupt: 

# Part 4: Check Portfolio Performance

In [ ]:
import alpaca_trade_api as tradeapi
import exchange_calendars as tc
import numpy as np
import pandas as pd
import pytz
import yfinance as yf
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from datetime import datetime as dt
from finrl.plot import backtest_stats
import matplotlib.pyplot as plt

In [ ]:
def get_trading_days(start, end):
    nyse = tc.get_calendar('NYSE')
    df = nyse.sessions_in_range(pd.Timestamp(start,tz=pytz.UTC),
                                pd.Timestamp(end,tz=pytz.UTC))
    trading_days = []
    for day in df:
        trading_days.append(str(day)[:10])

    return trading_days

def alpaca_history(key, secret, url, start, end):
    api = tradeapi.REST(key, secret, url, 'v2')
    trading_days = get_trading_days(start, end)
    df = pd.DataFrame()
    for day in trading_days:
        df = df.append(api.get_portfolio_history(date_start = day,timeframe='5Min').df.iloc[:78])
    equities = df.equity.values
    cumu_returns = equities/equities[0]
    cumu_returns = cumu_returns[~np.isnan(cumu_returns)]
    
    return df, cumu_returns

def DIA_history(start):
    data_df = yf.download(['^DJI'],start=start, interval="5m")
    data_df = data_df.iloc[48:]
    baseline_returns = data_df['Adj Close'].values/data_df['Adj Close'].values[0]
    return data_df, baseline_returns

## Get cumulative return

In [ ]:
df_erl, cumu_erl = alpaca_history(key=API_KEY, 
                                  secret=API_SECRET, 
                                  url=API_BASE_URL, 
                                  start='2021-10-19', #must be within 1 month
                                  end='2021-10-22') #change the date if error occurs


In [ ]:
df_djia, cumu_djia = DIA_history(start='2021-10-19')


[*********************100%***********************]  1 of 1 completed


In [ ]:
print(df_erl)


                           profit_loss  profit_loss_pct      equity
timestamp                                                          
2021-10-19 09:30:00-04:00     -9324.06        -0.009005  1026133.65
2021-10-19 09:35:00-04:00    -10106.67        -0.009761  1025351.04
2021-10-19 09:40:00-04:00     -8463.56        -0.008174  1026994.15
2021-10-19 09:45:00-04:00     -9907.60        -0.009568  1025550.11
2021-10-19 09:50:00-04:00     -8516.10        -0.008224  1026941.61
...                                ...              ...         ...
2021-10-22 15:35:00-04:00      1560.63         0.001507  1037018.34
2021-10-22 15:40:00-04:00       729.00         0.000704  1036186.71
2021-10-22 15:45:00-04:00       546.57         0.000528  1036004.28
2021-10-22 15:50:00-04:00       459.65         0.000444  1035917.36
2021-10-22 15:55:00-04:00       862.31         0.000833  1036320.02

[312 rows x 3 columns]


In [ ]:
print(df_djia)

                                   Open          High  ...     Adj Close    Volume
Datetime                                               ...                        
2021-10-19 13:30:00-04:00  35428.171875  35434.890625  ...  35434.699219   2301420
2021-10-19 13:35:00-04:00  35435.179688  35440.160156  ...  35437.269531   1806707
2021-10-19 13:40:00-04:00  35437.480469  35437.710938  ...  35430.851562   1797519
2021-10-19 13:45:00-04:00  35430.601562  35432.808594  ...  35423.058594   1917764
2021-10-19 13:50:00-04:00  35422.929688  35435.828125  ...  35435.691406   2018081
...                                 ...           ...  ...           ...       ...
2021-10-22 15:35:00-04:00  35708.800781  35710.929688  ...  35684.738281   3547180
2021-10-22 15:40:00-04:00  35684.339844  35692.960938  ...  35675.769531   4243899
2021-10-22 15:45:00-04:00  35675.710938  35680.800781  ...  35670.238281   5178920
2021-10-22 15:50:00-04:00  35670.808594  35699.308594  ...  35692.921875   9231104
2021

In [ ]:
df_erl.tail()

,profit_loss,profit_loss_pct,equity
timestamp,,,
2021-10-22 15:35:00-04:00,1560.63,0.001507,1037018.34
2021-10-22 15:40:00-04:00,729.00,0.000704,1036186.71
2021-10-22 15:45:00-04:00,546.57,0.000528,1036004.28
2021-10-22 15:50:00-04:00,459.65,0.000444,1035917.36
2021-10-22 15:55:00-04:00,862.31,0.000833,1036320.02


In [ ]:
returns_erl = cumu_erl -1 
returns_dia = cumu_djia - 1
returns_dia = returns_dia[:returns_erl.shape[0]]
print('len of erl return: ', returns_erl.shape[0])
print('len of dia return: ', returns_dia.shape[0])

len of erl return:  312
len of dia return:  264


## plot and save

In [ ]:
import matplotlib.pyplot as plt
plt.figure(dpi=1000)
plt.grid()
plt.grid(which='minor', axis='y')
plt.title('Stock Trading (Paper trading)', fontsize=20)
plt.plot(returns_erl, label = 'ElegantRL Agent', color = 'red')
#plt.plot(returns_sb3, label = 'Stable-Baselines3 Agent', color = 'blue' )
#plt.plot(returns_rllib, label = 'RLlib Agent', color = 'green')
plt.plot(returns_dia, label = 'DJIA', color = 'grey')
plt.ylabel('Return', fontsize=16)
plt.xlabel('Year 2021', fontsize=16)
plt.xticks(size = 14)
plt.yticks(size = 14)
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MultipleLocator(78))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(6))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.005))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1, decimals=2))
ax.xaxis.set_major_formatter(ticker.FixedFormatter(['','10-19','','10-20',
                                                    '','10-21','','10-22']))
plt.legend(fontsize=10.5)
plt.savefig('papertrading_stock.png')